<a href="https://colab.research.google.com/github/r42arty/hse/blob/main/mod5/LLM_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Установка LM studio

# 🧪 Практическое задание: Тестирование LLM в LM Studio

## 📥 Шаг 1: Установка LM Studio
1. Перейдите по ссылке: [lmstudio.ai](https://lmstudio.ai/)  
2. Скачайте установщик и установите приложение на свой компьютер.  
3. Запустите LM Studio.

---

## 🔍 Шаг 2: Загрузка моделей
LM Studio напрямую интегрирован с [HuggingFace](https://huggingface.co/) и умеет скачивать оттуда модели.

Рекомендуется попробовать:
- **LLaMA‑2** (разные размеры)  
- **Qwen**  
- **Vikhr** (русскоязычные)  
- **Mistral (Mistral‑7B, Mixtral‑MoE)**  
- **OpenChat / OpenHermes / MPT**  (openai/gpt-oss-20b)
- (дополнительно — можно поискать новые релизы на Hugging Face Hub)

---

## ⚙️ Шаг 3: Использование Power User режима
1. В левом нижнем углу интерфейса переключитесь с режима **User → Power User**.  
2. В этом режиме откроются дополнительные параметры:
   - Настройка температуры (`temperature`)  
   - Top‑p и top‑k  
   - Penalties (frequency / presence)  
   - Установка **системного промпта**  
   - Макс. длина ответа (`max_tokens`)

---

## 🧠 Шаг 4: Эксперименты
1. Составьте несколько вопросов **логического и простого уровня**:  
   - Простой: *"Объясни, что такое машинное обучение простыми словами."*  
   - Логический: *"У Пети было 5 яблок. Он отдал 2 другу и съел одно. Сколько осталось?"*  
2. Задайте эти вопросы **разным моделям**
3. Сравните ответы по критериям:  
   - Понятность и связность текста  
   - Наличие пошаговых рассуждений (reasoning)  
   - Склонность к «галлюцинациям»  
   - Разница между старыми и новыми моделями  

---

## 📝 Задание 1
1. Установите LM Studio и скачайте минимум **2 модели разных серий** (например, LLaMA и Vikhr).  
2. В режиме **Power User** протестируйте параметры генерации:  
   - `temperature` = 0.0, 0.7, 1.2  
   - `top_p` = 0.5, 0.9  
   - `presence_penalty` = 0.0, 0.8  
3. Сравните, как изменяется стиль и креативность текста.  
4. **Ответьте письменно**:
   - Какая модель показалась наиболее «разумной»?  
   - Как параметры влияли на разнообразие и точность ответов?  
   - Отличались ли модели с reasoning (например, Mistral‑8x7B / Qwen) от обычных (Vikhr‑Instruct)?  

---

## 🎯 Цель
- Освоить работу с LM Studio.  
- Научиться сравнивать модели с reasoning и без.  
- Понять, как параметры генерации влияют на стиль и точность текста.  
- Сформировать собственное мнение о том, какие модели лучше подходят для разных задач.



# Ответ

## Модели для экспериметнов
- Vikhrmodels/QVikhr-2.5-1.5B-Instruct-r_GGUF
- openai/gpt-oss-20b

## Промпт для тестирования
- Причины падения Римской империи

## 1. Какая модель показалась наиболее «разумной»?
Vikhr 12B Instruct — простые, уместные ответы на русском без «ухода» в фантастику.

### 2. Как параметры влияли на разнообразие и точность ответов?
- При низкой temperature и top_p получались скучные, односложные ответы.  
- Оптимум: temp=0.7, top_p=0.9 — баланс между креативностью и точностью.  
- Высокая temperature (1.2) давала креатив, но иногда «галлюцинации» (например, Симпсоны и инопланетяне).  

### 3. Отличались ли модели с reasoning от обычных?
- Да. Qwen чётко показывал шаги рассуждений (особенно про столицу и историю).  
- Vikhr и LLaMA-3 выдавали сразу готовый ответ, без пошагового анализа.  

# 📚 Домашнее задание: Сравнение текстов с помощью TF‑IDF и BERT

## 🎯 Цели задания
- Научиться представлять тексты в числовом виде (векторы).
- Попробовать два подхода: **TF‑IDF** и **BERT‑эмбеддинги**.
- Научиться измерять **сходство текстов**.


In [1]:
# Установка библиотек (в Colab HuggingFace уже часто стоит, но на всякий случай)
!pip install transformers

In [16]:
texts = [
    "Маркетинговая стратегия бренда основана на цифровых каналах и персонализированных рассылках.",
    "Для удержания клиентов компания применяет email‑маркетинг с динамическими предложениями и бонусами.",
    "Социальные сети помогают брендам повышать узнаваемость и формировать сообщество вокруг продукта.",
    "Контент‑маркетинг через статьи и блоги генерирует органический трафик и повышает доверие аудитории.",
    "Таргетированная реклама в поисковых системах позволяет привлекать релевантных пользователей в момент их запроса.",
    "Аналитика поведения клиентов помогает оптимизировать воронку продаж и повышать конверсию.",
    "Омниканальные коммуникации обеспечивают единый опыт пользователя во всех точках взаимодействия с брендом.",
    "Использование чат‑ботов улучшает качество клиентского сервиса и снижает нагрузку на call‑центр.",
    "Партнёрский маркетинг и коллаборации усиливают охват аудитории и формируют новые точки контакта.",
    "SEO‑оптимизация сайта повышает позиции в поисковых системах и привлекает трафик без дополнительных затрат."
]


## ✍️ Часть 1. TF‑IDF
1. Векторизуем тексты с помощью `TfidfVectorizer`.
2. Посчитаем косинусное сходство между ними.

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# векторизуем тексты с помощью TfidfVectorizer
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(texts)

# считаем косинусное сходство между ними
similarity_tfidf = cosine_similarity(X_tfidf, X_tfidf)

# выводим матрицу косинусного сходства (TF-IDF)
df_tfidf = pd.DataFrame(
    similarity_tfidf,
    index=[f"T{i+1}" for i in range(len(texts))],
    columns=[f"T{i+1}" for i in range(len(texts))]
)
df_tfidf.style.format("{:.3f}")

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10
T1,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.071,0.000,0.000
T2,0.000,1.000,0.000,0.058,0.000,0.082,0.000,0.000,0.060,0.000
T3,0.000,0.000,1.000,0.000,0.000,0.080,0.000,0.000,0.000,0.000
T4,0.000,0.058,0.000,1.000,0.000,0.000,0.000,0.000,0.134,0.140
T5,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.136
T6,0.000,0.082,0.080,0.000,0.000,1.000,0.000,0.000,0.000,0.000
T7,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000
T8,0.071,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000
T9,0.000,0.060,0.000,0.134,0.000,0.000,0.000,0.000,1.000,0.000
T10,0.000,0.000,0.000,0.140,0.136,0.000,0.000,0.000,0.000,1.000


In [18]:
# обнуляем диагональ
np.fill_diagonal(similarity_tfidf, 0)

# находим индексы максимального значения
i, j = np.unravel_index(similarity_tfidf.argmax(), similarity_tfidf.shape)

print(f"Наиболее похожие тексты по TF-IDF — индексы T{i+1} и T{j+1}, косинусное сходство = {similarity_tfidf[i, j]:.3f}:\n")
print(texts[i])
print(texts[j])

Наиболее похожие тексты по TF-IDF — индексы T4 и T10, косинусное сходство = 0.140:

Контент‑маркетинг через статьи и блоги генерирует органический трафик и повышает доверие аудитории.
SEO‑оптимизация сайта повышает позиции в поисковых системах и привлекает трафик без дополнительных затрат.


### 📝 Задание 2
- Найдите, какие два текста TF‑IDF считает наиболее похожими.
- Объясните, почему именно они схожи.

Есть пересечение слова «трафик», поэтому такие тексты он считает наиболее близкими

## ✍️ Часть 2. BERT‑эмбеддинги
1. Используйте модель из hugging face
2. Посчитаем косинусное сходство.


In [19]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

# Загружаем токенизатор и модель из Hugging Face
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Функция для получения эмбеддингов (среднее по токенам)
def get_embeddings(texts):
    # Токенизация
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        # Берём эмбеддинги скрытых состояний (last_hidden_state)
        embeddings = outputs.last_hidden_state
        # Усредняем токены по последовательности → получаем sentence embedding
        embeddings = embeddings.mean(dim=1)
    return embeddings.numpy()

# Получаем эмбеддинги
X_bert = get_embeddings(texts)

# считаем косинусное сходство
similarity_bert = cosine_similarity(X_bert, X_bert)

# в таблицу
df_bert = pd.DataFrame(
    similarity_bert,
    index=[f"T{i+1}" for i in range(len(texts))],
    columns=[f"T{i+1}" for i in range(len(texts))]
)
df_bert.style.format("{:.3f}")


,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10
T1,1.000,0.684,0.702,0.724,0.573,0.772,0.699,0.783,0.706,0.646
T2,0.684,1.000,0.544,0.600,0.369,0.684,0.632,0.552,0.614,0.501
T3,0.702,0.544,1.000,0.766,0.585,0.804,0.737,0.698,0.754,0.627
T4,0.724,0.600,0.766,1.000,0.541,0.696,0.701,0.697,0.852,0.620
T5,0.573,0.369,0.585,0.541,1.000,0.503,0.631,0.518,0.553,0.672
T6,0.772,0.684,0.804,0.696,0.503,1.000,0.778,0.665,0.726,0.600
T7,0.699,0.632,0.737,0.701,0.631,0.778,1.000,0.607,0.753,0.631
T8,0.783,0.552,0.698,0.697,0.518,0.665,0.607,1.000,0.663,0.533
T9,0.706,0.614,0.754,0.852,0.553,0.726,0.753,0.663,1.000,0.534
T10,0.646,0.501,0.627,0.620,0.672,0.600,0.631,0.533,0.534,1.000


In [23]:
# обнуляем диагональ
np.fill_diagonal(similarity_bert, 0)

# находим индексы максимального значения
i, j = np.unravel_index(similarity_bert.argmax(), similarity_bert.shape)

print(f"Наиболее похожие тексты по TF-IDF — индексы T{i+1} и T{j+1}, косинусное сходство = {similarity_bert[i, j]:.3f}:\n")
print(texts[i])
print(texts[j])

Наиболее похожие тексты по TF-IDF — индексы T4 и T9, косинусное сходство = 0.852:

Контент‑маркетинг через статьи и блоги генерирует органический трафик и повышает доверие аудитории.
Партнёрский маркетинг и коллаборации усиливают охват аудитории и формируют новые точки контакта.


### 📝 Задание 3
- Сравните матрицу сходства TF‑IDF и BERT.
- Какие пары текстов модель BERT находит более похожими?  
- Почему BERT в этом случае "понимает смысл" лучше?


**TF-IDF** считает наиболее похожими тексты **T4** и **T10** — сходство основано на совпадении одинаковых слов «трафик»

**BERT** считает наиболее похожими тексты **T4** и **T9** — сходство основано на «смысле» использования стратегии по увеличению охвата пользователей

**Почему BERT "понимает смысл" лучше:**
- TF-IDF работает только с частотой слов
- BERT строит семантические векторные представления, где близкие по смыслу фразы оказываются рядом даже без прямых совпадений

# ⚙️ Основные параметры генерации текста (GPT‑модели)

При запросе к модели можно управлять её поведением через специальные параметры.  

---

### 🔧 Параметры

- **temperature = 0.8**  
  - Управляет степенью «креативности» модели.  
  - Чем выше значение (до 2.0), тем более разнообразные и неожиданные ответы.  
  - Чем ниже (например 0.2), тем ответы более предсказуемые, формальные.  

---

- **max_tokens = 400**  
  - Ограничивает максимальное количество токенов (слов/частей слов), которое модель сгенерирует в ответе.  
  - Позволяет контролировать длину текста.  

---

- **top_p = 0.9** (называется «nucleus sampling»)  
  - Альтернатива `temperature`.  
  - Модель выбирает токены не из всех возможных, а только из «лучших», суммарной вероятностью 90%.  
  - При `top_p=1.0` используется полный набор без ограничений.  

---

- **presence_penalty = 0.6**  
  - Наказывает модель за **повторение одинаковых идей/тем**.  
  - Чем выше число, тем больше модель старается вносить что‑то новое.  

---

- **frequency_penalty = 0.4**  
  - Наказывает модель за **частое повторение одних и тех же слов**.  
  - Чем выше, тем разнообразнее словарный запас в ответе.  

---

- **n = 1**  
  - Количество разных вариантов ответа за один запрос.  
  - Если поставить 3, модель вернёт 3 разных завершения.  

---

- **stream = False**  
  - Управляет тем, как возвращается ответ:  
    - `False` → вернуть всё сразу.  
    - `True` → потоковая генерация (можно отображать текст по кусочкам, как в чате).  

---

## 🎓 Итог
- `temperature` и `top_p` регулируют «креативность» и разнообразие.  
- `max_tokens` ограничивает длину ответа.  
- `presence_penalty` и `frequency_penalty` уменьшают повторы.  
- `n` позволяет получить несколько вариантов ответа.  
- `stream` управляет режимом выдачи (сразу/потоком).  


### Реализовать параметры генерации

# 🧪 Задание 4: Реализация параметров генерации токенов

В этом задании нужно **самостоятельно реализовать функции** изменения распределения вероятностей, исходя из описаний параметров генерации языковых моделей.

## 📌 Дано:
Пример логитов выходного слоя:

```python
import numpy as np

logits = np.array([3.0, 2.5, 2.0, 1.5, 1.0, 0.5, 0.0, -0.5, -1.0, -1.5])
```
### В ответ приложите дописанный кусок кода
```python
# перевод логитов в вероятность
probs = # ваш код
def apply_temperature(probs, temperature=1.0):
    """
    Масштабирует распределение вероятностей по температуре.
    Чем ниже T < 1 → распределение более "острое",
    Чем выше T > 1 → распределение выравнивается.
    """
    # ваш код

def apply_top_k(probs, k=10):
    """
    Оставляет только k наиболее вероятных элементов,
    остальные вероятности обнуляются и распределение нормализуется.
    """
    # ваш код

def apply_top_p(probs, p=0.9):
    """
    Реализация nucleus sampling.
    Оставляем минимальный набор токенов,
    суммарная вероятность которых ≥ p.
    Остальные обнуляем и нормализуем распределение.
    """
    # ваш код

def sample_from_probs(probs):
    """
    Выбор токена случайным образом в соответствии с вероятностным распределением.
    """
    return int(np.random.choice(len(probs), p=probs))


In [24]:
import numpy as np

logits = np.array([3.0, 2.5, 2.0, 1.5, 1.0, 0.5, 0.0, -0.5, -1.0, -1.5])

In [29]:
# перевод логитов в вероятность

def softmax(logits):
    z = logits - np.max(logits)   # для численной стабильности
    e = np.exp(z)
    return e / e.sum()

probs = softmax(logits)

def apply_temperature(probs, temperature=1.0):
    """
    Масштабирует распределение вероятностей по температуре.
    Чем ниже T < 1 → распределение более "острое",
    Чем выше T > 1 → распределение выравнивается.
    """
    if temperature <= 0:
        raise ValueError("temperature must be > 0")
    if temperature == 1.0:
        return probs
    scaled = probs ** (1.0 / temperature)
    scaled /= scaled.sum()
    return scaled

def apply_top_k(probs, k=10):
    """
    Оставляет только k наиболее вероятных элементов,
    остальные вероятности обнуляются и распределение нормализуется.
    """
    k = max(1, min(k, probs.size))
    # индексы k крупнейших
    idx = np.argpartition(probs, -k)[-k:]
    mask = np.zeros_like(probs, dtype=bool)
    mask[idx] = True
    filtered = np.where(mask, probs, 0.0)
    filtered /= filtered.sum()
    return filtered

def apply_top_p(probs, p=0.9):
    """
    Реализация nucleus sampling.
    Оставляем минимальный набор токенов,
    суммарная вероятность которых ≥ p.
    Остальные обнуляем и нормализуем распределение.
    """
    order = np.argsort(probs)[::-1]         # сортировка по убыванию
    sorted_probs = probs[order]
    cumsum = np.cumsum(sorted_probs)
    cut = np.searchsorted(cumsum, p) + 1    # минимальный порог
    keep_idx = order[:cut]
    mask = np.zeros_like(probs, dtype=bool)
    mask[keep_idx] = True
    filtered = np.where(mask, probs, 0.0)
    filtered /= filtered.sum()
    return filtered

def sample_from_probs(probs):
    """
    Выбор токена случайным образом в соответствии с вероятностным распределением.
    """
    return int(np.random.choice(len(probs), p=probs))

# вывод
print("Базовое распределение:", [format(x, ".3f") for x in probs])
print("Температура 0.5:", [format(x, ".3f") for x in apply_temperature(probs, 0.5)])
print("Top-k=3:", [format(x, ".3f") for x in apply_top_k(probs, 3)])
print("Top-p=0.9:", [format(x, ".3f") for x in apply_top_p(probs, 0.9)])
print("Сэмплированный токен:", sample_from_probs(probs))

Базовое распределение: ['0.396', '0.240', '0.146', '0.088', '0.054', '0.033', '0.020', '0.012', '0.007', '0.004']
Температура 0.5: ['0.632', '0.233', '0.086', '0.031', '0.012', '0.004', '0.002', '0.001', '0.000', '0.000']
Top-k=3: ['0.506', '0.307', '0.186', '0.000', '0.000', '0.000', '0.000', '0.000', '0.000', '0.000']
Top-p=0.9: ['0.429', '0.260', '0.158', '0.096', '0.058', '0.000', '0.000', '0.000', '0.000', '0.000']
Сэмплированный токен: 5


#### Проверочное задание

In [32]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM

# ---------- Загружаем маленькую модель GPT ----------
model_name = "ai-forever/rugpt3small_based_on_gpt2"  # урезанный GPT-2
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Переведем модель на GPU если доступно
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


# ---------- Реализация генерации ----------
def generate_text(prompt, max_len=30, temperature=1.0, top_k=None, top_p=None):
    # Токенизируем
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    tokens = input_ids.tolist()[0]

    for _ in range(max_len):
        # Получаем логиты модели
        with torch.no_grad():
            outputs = model(torch.tensor([tokens]).to(device))
        logits = outputs.logits[0, -1, :].cpu().numpy()

        # Переводим в вероятности
        exp_logits = np.exp(logits - np.max(logits))
        probs = exp_logits / exp_logits.sum()

        # Temperature
        if temperature != 1.0:
            probs = apply_temperature(probs, temperature)

        # Top-k
        if top_k is not None:
            probs = apply_top_k(probs, top_k)

        # Top-p
        if top_p is not None:
            probs = apply_top_p(probs, top_p)

        # Сэмплируем токен
        next_token = sample_from_probs(probs)
        tokens.append(next_token)

        # Если дошли до конца — выходим
        if next_token == tokenizer.eos_token_id:
            break

    return tokenizer.decode(tokens)


# ---------- Пробуем! ----------
print("=== Обычная генерация (temperature=1) ===\n")
print(generate_text("В холодную зимнюю пору ", temperature=1.0, max_len=30))

print("\n=== Жёсткий greedy (temperature=0.5, top-k=5) ===\n")
print(generate_text("В холодную зимнюю пору ", temperature=0.5, top_k=5, max_len=30))

print("\n=== Креативная (temperature=1.5, top-p=0.9) ===\n")
print(generate_text("В холодную зимнюю пору ", temperature=1.5, top_p=0.9, max_len=30))


=== Обычная генерация (temperature=1) ===

В холодную зимнюю пору  весь город облазил, везде намокая от мороза(прекрасно снимал).  Другой съемочный отряд заглянул в метро и был пофот

=== Жёсткий greedy (temperature=0.5, top-k=5) ===

В холодную зимнюю пору 
А.С.Пушкин
























=== Креативная (temperature=1.5, top-p=0.9) ===

В холодную зимнюю пору  задушить  серый санинтРНемедленно употреблять мимино шарикиВатная грудь &#1072; часть ладониИгровые вихры выпуск


# Практика Вызов АПИ

# 📝 Задание: использование LLM через API

## Зачем это нужно?
Используя API, можно напрямую работать с моделями вроде ChatGPT, DeepSeek, Qwen и др.  
Это позволяет:
- делать запросы программно (без чата в интерфейсе),
- автоматизировать работу (интеграция в скрипты, сервисы),
- управлять параметрами вывода (длина ответа, стиль и др.).

---

## 🔑 Что такое токен?
- **Токен** — это уникальный ключ (идентификатор), который позволяет авторизоваться при обращении к API.  
- Его можно получить в личном кабинете на сайте хоста (ChatGPT, DeepSeek, Qwen и т. д.).  
- Обычно нужно войти в аккаунт и, возможно, подключить платёж (например, в DeepSeek можно просто пополнить баланс на несколько долларов).  
- 🔎 Если нет зарубежной карты — можно поискать способы оплаты (например, [см. пост на Хабре](https://habr.com/ru/articles/895864/)).

⚠️ Важно: токен нужно хранить в секрете и не публиковать открыто.  
Обычно используют `.env`‑файл или переменные окружения, чтобы спрятать токен «под капот».  

---

## 🌐 Полезные ссылки
- **API DeepSeek:** https://platform.deepseek.com/usage  
- **API OpenAI (ChatGPT):** https://platform.openai.com  
- **Qwen API (Alibaba Cloud):** https://www.modelscope.cn  

---

## ✅ Задание
1. Зарегистрироваться на одной из платформ (ChatGPT / DeepSeek / Qwen).  
2. Получить токен в личном кабинете.  
3. Сделать первый API‑запрос: например, "Объясни простыми словами, что такое API".  
4. Спрятать токен (через `.env` или переменные окружения).  
5. Попробовать изменить параметры запроса (например, длину ответа, температуру).  

---


In [ ]:
import os
from getpass import getpass
os.environ["API_KEY"] = getpass("Введите ключ : ")

Введите ключ : ··········


После загрузки токена, можно приступать к генерации.  
В зависисмости от того, какой моделью пользуетесь, схема запроса может меняться. Обычно есть примеры в сети или в доке.  
Нужно изменить base_url и model, если будете пользоваться не deepseek

In [ ]:
import os
from openai import OpenAI

# Предполагается, что у вас в окружении заданы:
# os.environ["DEEPSEEK_API_KEY"] = "ваш_ключ"
# (например через `os.environ` или getpass)

# при создании клиента передаём base_url и key
client = OpenAI(
    api_key=os.environ.get("API_KEY"),
    base_url="https://api.deepseek.com"  # URL DeepSeek API
)

# выполняем чат-запрос
response = client.chat.completions.create(
    model="deepseek-chat",  # у DeepSeek сейчас основная модель так называется
    messages=[
        {"role": "system", "content": "Ты — дружелюбный эксперт по истории искусства."},
        {"role": "user", "content": "Расскажи о влиянии импрессионизма на современную визуальную культуру простым и интересным языком."}
    ],
    temperature=0.8,
    max_tokens=400,
    top_p=0.9,
    presence_penalty=0.6,
    frequency_penalty=0.4,
    n=1,
)

# выводим текcт ответа
print(response.choices[0].message.content)


Конечно! Давайте представим, что импрессионизм — это как дедушка современной визуальной культуры. Он хоть и родился в XIX веке, но его ДНК до сих пор живёт во всём, что мы видим вокруг — от фильмов и рекламы до селфи в Instagram. 😊  

Вот несколько ключевых моментов, почему импрессионизм так важен сегодня:  

### 1. **«Лови момент!»**  
Импрессионисты первыми начали рисовать не «идеальные» сцены, а сиюминутные впечатления: солнечный блик на воде, ветер в листве, улыбку человека. Это как предтеча сторис в соцсетях — мы тоже любим фиксировать мгновения, пусть и не идеальные, но живые.  

### 2. **Свет и цвет — главные герои**  
Художники вроде Моне или Ренуара показали, что тень может быть синей, а снег — розовым на закате. Современная фотография, кино и даже фильтры в приложениях (например, те самые тёплые «вечерние» тона) во многом обязаны этому открытию.  

### 3. **Демократизация искусства**  
Импрессионисты рисовали обычных людей: гуляющих в парке, пьющих кофе, танцующих. Не богов и

## Задание 5 - попробовать различные параметры генерации на внешней моделе

### 🟦 Базовый контроль
- `temperature = 0.0`  
- `top_p = 1.0`  
- `max_tokens = 100`  

---

### 🟩 Творчество
- `temperature = 1.0`  
- `top_p = 0.9`  
- `max_tokens = 250`  

---

### 🟨 Ограничение длины
- `temperature = 0.7`  
- `top_p = 0.9`  
- `max_tokens = 50`  

---

### 🟥 Штраф на повторы
- `temperature = 0.7`  
- `top_p = 0.9`  
- `frequency_penalty = 0.8`  
- `presence_penalty = 0.5`  

---

### 🟪 Несколько вариантов (n = 3)
- `temperature = 0.9`  
- `top_p = 0.95`  
- `n = 3`  
- `max_tokens = 150`  


### Ответы
   
#### Исходный промт
ваш ответ.

#### Модель
ваш ответ.  

#### 🟦 Базовый контроль
ваш ответ  

#### 🟩 Творчество
ваш ответ.

#### 🟨 Ограничение длины
ваш ответ.

#### 🟥 Штраф на повторы
ваш ответ  

#### 🟪 Несколько вариантов
ваш ответ